In [1]:
import numpy as np
import itertools
from pylatex import Document, Section, Subsection, Tabular
from pylatex import Math, TikZ, Axis, Plot, Figure, Matrix, Alignat
from pylatex.utils import italic
import os
from time import sleep
import cmath as m

geometry_options = {"tmargin": ".5cm", "lmargin": ".5cm"}
doc = Document(geometry_options=geometry_options)
try:
    os.remove("./LatexCode.tex")
except:
    print("no file to remove")
sleep(1)

In [2]:
class LayerOf:
    LayerOutputVector = []
    def __init__(self, NumberOfQubit):
        self.NumberOfQubit = NumberOfQubit

    def compute(self, ipVec, gateList):
        # if len(ipVec) != self.NumberOfQubit:
        if ipVec.size != 2**self.NumberOfQubit:
            raise Exception("Incorrect Input Vector")
        if len(gateList) != self.NumberOfQubit:
            raise Exception("Number of Gates are not compatible to circuit")

        if 'cx' in gateList:   # Reversing the list because in tensor implementation for CNOT is in reverse
            gateList.reverse()

        firstGate = gateList[0]
        flag = 0
        finalGateMat = None
        ctrl = []
        target = 0
        isctrlnot = 1
        lam = 0
        theta = 0
        if 'c' in gateList:    # For control and target computation (Limited to CNOT)
            for i, gate in enumerate(gateList):
                if gate == 'c':
                    ctrl.append(i)
                if gate == 'cx':
                    target = i
            
        
        for x in gateList:
            if 'rx' in x:
                theta = int(x[3:len(x)-1])
            elif 'ry' in x:
                theta = int(x[3:len(x)-1])
            elif 'rz' in x:
                lam = int(x[3:len(x)-1])
                
        if len(gateList) == 1:      # For One Qubit Circuit
            if 'r' in firstGate:
                finalGateMat = self.tensor(firstGate, None, flag, lam, theta)
            else:
                finalGateMat = self.tensor(firstGate, None, flag, ctrl, target)            
        for gate in gateList[1:]:
            if gate == 'c' and firstGate != 'cx':
                flag =1
                continue
            elif firstGate == 'cx':
                firstGate = None
                flag = 1
                isctrlnot = 2
            if gate == 'cx':
                isctrlnot = 2
            if flag == 0 and isctrlnot != 2:   # for first loop iteration both gates are in str format
                if 'r' in gate:
                    finalGateMat = self.tensor(firstGate, gate, flag, lam, theta)
                else:
                    finalGateMat = self.tensor(firstGate, gate, flag, ctrl, target)
                flag +=1
                
            elif flag == 0 and firstGate == 'i':   # for first loop iteration both gates are in str format
                print(flag, isctrlnot, firstGate)
                if 'r' in gate:
                    finalGateMat = self.tensor(firstGate, gate, flag, lam, theta)
                else:
                    finalGateMat = self.tensor(firstGate, gate, flag, ctrl, target)
                firstGate = None
                
            else:
                if 'r' in gate:
                    finalGateMat = self.tensor(gate, finalGateMat, flag, lam, theta)
                else:
                    finalGateMat = self.tensor(gate, finalGateMat, isctrlnot, ctrl, target)
                isctrlnot = 1

        # print(finalGateMat, finalGateMat.shape)
        LayerOutputVector = np.matmul(ipVec, finalGateMat)
        gList = np.array(gateList)
        return gList, ipVec, finalGateMat, LayerOutputVector
    

    def tensor(self, g1, g2, flag = 1, *extraParam):
        if flag == 1:        # for General Call
            return np.kron(self.matrixGenerator(self.NumberOfQubit, g1, extraParam[0], extraParam[1]), g2)
        elif flag == 2:      # for mcnot
            return self.matrixGenerator(self.NumberOfQubit, 'cx', extraParam[0], extraParam[1])
        else:       # for First Call
            if g2 == None:
                return self.matrixGenerator(self.NumberOfQubit, g1, extraParam[0], extraParam[1])
            else:   #both gates are in str format
                return np.kron(self.matrixGenerator(self.NumberOfQubit, g2, extraParam[0], extraParam[1]),
                               self.matrixGenerator(self.NumberOfQubit, g1, extraParam[0], extraParam[1]))
        
    
    def printConvention(self):
        print("#  bin value format       => [q0 q1 q2 . . .]")
        print("#  respective gate format => [G0 G1 G2 . . .] \n")

        
    def matrixGenerator(self, n, GateType, *gateParam): #gateParam[0] = controls; gateParam[1] = target
        if GateType == 'h':                             #gateParam[0] = lamda; gateParam[1] = theta
            hM = np.matrix('0.707 0.707; 0.707 -0.707')
            return hM

        if GateType == 'x':
            xM = np.matrix('0 1; 1 0')
            return xM

        if GateType == 'z':
            xM = np.matrix('1 0; 0 -1')
            return xM
        
        if GateType == 'i':
            xM = np.matrix('1 0; 0 1')
            return xM

        if GateType == 'cx':
            control = gateParam[0]                # qubit number
            target = gateParam[1]                 # qubit number
            
            if target in control:
                raise Exception ("Target and Control bit is same")

            allgreater = 1
            allsmall = 1
            for x in control:
                if x < target:
                    allgreater = 0
                    break
            for x in control:
                if x > target:
                    allsmall = 0
                    break

            if allgreater + allsmall != 0:
                if allgreater == 0:
                    target = target - min(control)
                    control = [x - min(control) for x in control]

                if allsmall == 0:
                    control = [x - target for x in control]
                    target = 0

            
            sqs = [''.join(s) for s in list(itertools.product(*[['0', '1']] * (len(control)+1)))]           
            ccnotArr = np.zeros((2**len(sqs[0]), 2**len(sqs[0])))
            for i, binSeq in enumerate(sqs):
                Flag = 0                # 1 means control active; 0 means control fail
                for c in control:
                    if binSeq[c] == '1':
                        Flag = 1
                    else:
                        Flag = 0
                        break
                if Flag == 0:
                    ccnotArr[i][i] = 1
                else:
                    s = ''
                    if binSeq[target] == '1':
                        for pos, ch in enumerate(binSeq):
                            if pos == target:
                                ch = '0'
                            s = s+ch
                    else:
                        for pos, ch in enumerate(binSeq):
                            if pos == target:
                                ch = '1'
                            s = s+ch
                    r = sqs.index(s)
                    ccnotArr[r][i] = 1           
            return ccnotArr
        
        if GateType[:2] == 'rz':
            lam = gateParam[0]
            M = np.matrix([[m.exp(-lam*1j), 0], [0, m.exp(lam*1j)]])
            return M
        
        if GateType[:2] == 'rx':
            th = gateParam[1]
            M = np.matrix([[m.cos(th/2), -1j*m.sin(th/2)], [-1j*m.sin(th/2), m.cos(th/2)]])
            return M
        
        if GateType[:2] == 'ry':
            th = gateParam[1]
            M = np.matrix([[m.cos(th/2), -1*m.sin(th/2)], [m.sin(th/2), m.cos(th/2)]])
            return M

# Python to Latex Functions

In [3]:
def write(docObj, string):
    doc.append(Section(string))
    
def saveOutput(docObj, GateSymbol, GateM, iv, result, LayerNum, div = 1):
    iv = np.split(iv, div, 1)
    result = np.split(result, div, 1)
    with docObj.create(Subsection('Layer {}'.format(LayerNum))):
        doc.append("#  bin value format       => [q0 q1 q2 . . .]\n")
        doc.append(GateSymbol)
        for i, result in enumerate(result):
            docObj.append(Math(data=[Matrix(np.around(GateM, 2)), Matrix(np.around(iv[i], 2).T), '=', 
                                 Matrix(np.around(result, 2).T)]))

def generateLatexCode(docObj):
    try:
        docObj.generate_pdf('LatexCode')
    except:
        print("No Latex Compiler found on system: use overleaf")
    docObj.clear

# 

# <center>------------------ Circuit Implementation ---------------------</center>
#### Absence of gate must have 'i': (i: identity gate)
#### In control not gate controls and target representation will be 'c' and 'cx' respectively
#### Rotation gate format: rz[angle] (Ex. rx[90])
###### gateList, iv, FGateM, result = l1.compute(ipVec, ['x', 'x', 'i'])
###### gateList, iv, FGateM, result = l1.compute(result, ['i', 'i', 'i', 'rx[90]', 'i', 'i'])
###### gateList, iv, FGateM, result = l1.compute(ipVec, ['c', 'cx', 'i'])

#### saveOutput function div argument can be use to split the large matrix

#

In [4]:
n = 4
l1 = LayerOf(n)
l2 = LayerOf(n) #single layer object is sufficient

l1.printConvention()

#  bin value format       => [q0 q1 q2 . . .]
#  respective gate format => [G0 G1 G2 . . .] 



In [5]:
ipVec = np.zeros(2**n)
ipVec2 = np.zeros(2**n)

ipVec[0] = 1  # initializing to /0> (ket 0)
ipVec2[(2**0)-1] = 1  # initializing to /0> (ket 0)

ipVec = np.matrix(ipVec)
ipVec2 = np.matrix(ipVec2)

write(doc, "Statevector Comparator Circuit")

gateList, iv, FGateM, result = l1.compute(ipVec,
                                          ['rx[10]',
                                           'rx[11]',
                                           'i',
                                          'i'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 1, div = 1)

gateList, iv, FGateM, result = l1.compute(result, ['rz[12]',
                                                   'rz[13]',
                                                   'i',
                                                  'i'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 2, div = 1)

gateList, iv, FGateM, result = l1.compute(ipVec2, ['i',
                                                  'i',
                                                  'rx[10]',
                                                  'rx[11]'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 1, div = 1)

gateList, iv, FGateM, result = l1.compute(result, ['i',
                                                   'i',
                                                   'rz[12]',
                                                   'rz[13]'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 2, div = 1)


write(doc, "Part 2: Statevector Comparator Circuit")

gateList, iv, FGateM, result = l1.compute(ipVec, ['rx[1]',
                                                  'rx[2]',
                                                  'rx[10]',
                                                  'rx[11]'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 1, div = 1)

gateList, iv, FGateM, result = l1.compute(result, ['rz[1]',
                                                   'rz[2]',
                                                   'rz[10]',
                                                   'rz[11]'])
saveOutput(doc, gateList, [], iv, result, LayerNum = 2, div = 1)



# ========================================
generateLatexCode(doc)

No Latex Compiler found on system: use overleaf


In [6]:
# ipVec = np.zeros(2**n)
# ipVec[0] = 1  # initializing to /0> (ket 0)
# ipVec = np.matrix(ipVec)

# write(doc, "Statevector Comparator Circuit")

# gateList, iv, FGateM, result = l1.compute(ipVec, ['ry[1]', 'ry[2]', 'ry[3]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 1, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['rz[4]', 'rx[5]', 'ry[6]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 2, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['c', 'cx', 'i'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 3, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['c', 'i', 'cx'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 4, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['i', 'c', 'cx'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 5, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['ry[7]', 'ry[8]', 'ry[9]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 6, div = 1)

# gateList, iv, FGateM, result = l1.compute(result, ['rz[10]', 'rx[11]', 'ry[12]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 7, div = 1)

# # ========================================

# ateList, iv, FGateM, result = l2.compute(ipVec, ['ry[1]', 'ry[2]', 'ry[3]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 1, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['rz[4]', 'rx[5]', 'ry[6]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 2, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['c', 'cx', 'i'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 3, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['c', 'i', 'cx'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 4, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['i', 'c', 'cx'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 5, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['ry[7]', 'ry[8]', 'ry[9]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 6, div = 1)

# gateList, iv, FGateM, result = l2.compute(result, ['rz[10]', 'rx[11]', 'ry[12]'])
# saveOutput(doc, gateList, [], iv, result, LayerNum = 7, div = 1)

# generateLatexCode(doc)